In [ ]:
import numpy as np
import pandas as pd
import os
from tqdm.notebook import tqdm # Progress bars
import gc # Garbage collection for memory management
from copy import deepcopy
import sys

# Import individual modules to avoid circular imports
from scripts.utils import dataset, camera, image, metric, submission
from scripts.features import extraction, clustering

In [ ]:
DATA_DIR = "../data/image-matching-challenge-2025"
OUTPUT_FILE = "output_graph.csv" # Output file for results
TRAIN_DIR = os.path.join(DATA_DIR, "train")


# --- Feature Extraction Parameters ---
# Options: 'SIFT', 'AKAZE', 'ORB' (DISK/ALIKED need external setup)
FEATURE_EXTRACTOR_TYPE = 'SIFT'
SIFT_NFEATURES = 8000 # Max features per image for SIFT

# --- Matching Parameters ---
MATCHER_TYPE = 'FLANN' # 'BF' (Brute Force) or 'FLANN' (Fast Library for Approximate Nearest Neighbors)
LOWE_RATIO_TEST_THRESHOLD = 0.8 # For filtering good matches (knnMatch ratio)
MIN_INLIER_MATCHES_INITIAL = 15 # Min inliers for initial pairwise geometry check
MIN_INLIER_MATCHES_GRAPH = 10 # Min inliers to add edge to view graph (can be lower)

# --- Geometric Verification (RANSAC for Fundamental Matrix) ---
RANSAC_THRESHOLD = 1.5 # RANSAC reprojection threshold in pixels for findFundamentalMat

# --- Clustering Parameters ---
# Options: 'ConnectedComponents', 'Spectral'
CLUSTERING_ALGORITHM = 'ConnectedComponents'
MIN_CLUSTER_SIZE = 3 # Minimum images to form a valid scene cluster

# --- SfM Parameters ---
MIN_VIEWS_FOR_TRIANGULATION = 2 # Need at least two views for triangulation
PNP_RANSAC_THRESHOLD = 5.0 # RANSAC reprojection threshold for solvePnPRansac
PNP_CONFIDENCE = 0.999 # Confidence for PnPRansac
MIN_3D_POINTS_FOR_PNP = 6 # Minimum 3D points required for PnP

# --- Camera Intrinsics (Approximation - Not submitted, but needed for E/PnP) ---
# We estimate a default K matrix. Real K varies per image, but this is a common
# simplification if intrinsics aren't provided or estimated.
# Focal length is often approximated based on image width.
DEFAULT_FOCAL_LENGTH_FACTOR = 1.2
# Assuming cx, cy are image center. Will be calculated per image later.

print(f"Constants defined. Using {FEATURE_EXTRACTOR_TYPE} features and {MATCHER_TYPE} matcher.")
print(f"Data Directory: {DATA_DIR}")

In [ ]:
# Load the dataset
samples = dataset.load_dataset(DATA_DIR)

for dataset_name in samples:
    print(f'Dataset "{dataset_name}" -> num_images={len(samples[dataset_name])}')

In [ ]:
# Import matcher here to avoid circular import issues
from scripts.features import matching

def process_dataset(dataset_id, test_image_dir, predictions, extractor, matcher):
    """Runs the full pipeline for a single dataset."""
    print(f"\n--- Processing Dataset: {dataset_id} ---")

    dataset_path = os.path.join(test_image_dir, dataset_id)
    filename_to_index = {p.filename: idx for idx, p in enumerate(predictions)}
    # 1. Extract Features
    extracted_features, image_dims = extraction.load_and_extract_features_dataset(dataset_id, test_image_dir, extractor)
    image_ids_in_dataset = list(extracted_features.keys())

    if not extracted_features:
        print(f"No extracted_features extracted for dataset {dataset_id}. Marking all as outliers.")
        # Use image list from directory listing if extracted_features is empty but dir exists
        all_images = list(f.name for f in dataset_path.glob('*.png')) + \
                    list(f.name for f in dataset_path.glob('*.jpg')) + \
                    list(f.name for f in dataset_path.glob('*.jpeg'))
        for img_id in all_images:
            r_str, t_str = camera.format_pose(None, None)
            prediction_index = filename_to_index[img_id]
            predictions[prediction_index].cluster_index = "outliers"
            predictions[prediction_index].rotation = deepcopy(r_str)
            predictions[prediction_index].translation = deepcopy(t_str)
        return predictions


    # Add images found in directory but failed extraction to image_ids_in_dataset
    all_images_found = list(image_dims.keys())
    image_ids_set = set(image_ids_in_dataset)
    for img_id in all_images_found:
        if img_id not in image_ids_set:
            image_ids_in_dataset.append(img_id)


    # 2. Build View Graph
    G, pairwise_matches = clustering.build_view_graph(image_ids_in_dataset, extracted_features, matcher)


    # 3. Cluster Images
    clusters, outliers = clustering.cluster_images(G, algorithm=CLUSTERING_ALGORITHM, min_cluster_size=MIN_CLUSTER_SIZE)

    # 4. Process Outliers
    print(f"Marking {len(outliers)} images as outliers.")
    for img_id in outliers:
        r, t = camera.format_pose(None, None)
        prediction_index = filename_to_index[img_id]
        predictions[prediction_index].cluster_index = "outliers"
        predictions[prediction_index].rotation = r
        predictions[prediction_index].translation = t

    # 5. Run SfM per Cluster
    print(f"Running SfM for {len(clusters)} clusters...")
    for i, cluster_nodes in enumerate(clusters):
        cluster_label = f"cluster{i+1}"
        print(f"\nProcessing {cluster_label} ({len(cluster_nodes)} images)...")

        # Filter extracted_features/dims/matches for the current cluster
        cluster_features = {img_id: extracted_features[img_id] for img_id in cluster_nodes if img_id in extracted_features}
        cluster_dims = {img_id: image_dims[img_id] for img_id in cluster_nodes if img_id in image_dims}
        # Filter pairwise matches (tricky, need both nodes in cluster)
        cluster_pairwise_matches = {}
        for (id1, id2), matches in pairwise_matches.items():
             if id1 in cluster_nodes and id2 in cluster_nodes:
                 cluster_pairwise_matches[(id1, id2)] = matches


        cluster_poses = camera.estimate_poses_for_cluster(
            cluster_nodes,
            cluster_features,
            cluster_dims,
            matcher,
            cluster_pairwise_matches # Pass filtered matches
        )

        # Add results for this cluster
        for img_id in cluster_nodes:
            R, T = cluster_poses.get(img_id, (None, None)) # Get pose, default to None if not found
            r, t = camera.format_pose(R, T)
            prediction_index = filename_to_index[img_id]
            predictions[prediction_index].cluster_index = cluster_label
            predictions[prediction_index].rotation = deepcopy(r)
            predictions[prediction_index].translation = deepcopy(t)

        # Clean up memory
        del cluster_features, cluster_dims, cluster_poses, cluster_pairwise_matches
        gc.collect()

    print(f"--- Finished Processing Dataset: {dataset_id} ---")
    return predictions

In [ ]:
# Process train datasets
if os.path.isdir(TRAIN_DIR):
    train_datasets = [os.path.basename(os.path.join(TRAIN_DIR, d)) for d in os.listdir(TRAIN_DIR) if os.path.isdir(os.path.join(TRAIN_DIR, d))]
    # sort train datasets based on their number of files
    train_datasets.sort(key=lambda x: len(os.listdir(os.path.join(TRAIN_DIR, x))), reverse=False)
    print("=== Processing Train Datasets ===")
    extractor = extraction.get_feature_extractor('SIFT', SIFT_NFEATURES)
    matcher = matching.get_matcher('FLANN', 'SIFT')
    for dataset_name in train_datasets[:3]:
        samples[dataset_name] = process_dataset(dataset_name, TRAIN_DIR, samples[dataset_name], extractor, matcher)
else:
    print("Train directory not found")

In [ ]:
# Create a submission file.
submission.create_submission_file(samples, OUTPUT_FILE)

!head {OUTPUT_FILE}

In [ ]:
# Compute results if running on the training set.
# Don't do this when submitting a notebook for scoring. All you have to do is save your submission to /kaggle/working/submission.csv.

final_score, dataset_scores = metric.score(
    gt_csv=os.path.join(DATA_DIR, "train_labels.csv"),
    user_csv=OUTPUT_FILE,
    thresholds_csv=os.path.join(DATA_DIR, "train_thresholds.csv"),
    mask_csv=None,
    inl_cf=0,
    strict_cf=-1,
    verbose=True,
)